In [1]:
#install nba package

In [2]:
pip install nba_api

Note: you may need to restart the kernel to use updated packages.


## Python Libraries and NBA API Endpoints

In [3]:
import pandas as pd 
import numpy as np

from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import leaguegamefinder

## Pull In Playoff Shooting Data and Game Data to Win/Loss

In [4]:
import json 
import csv

def playoffs_shot_selection(season):
    shot_chart_data = shotchartdetail.ShotChartDetail(team_id = 0, season_type_all_star = 'Playoffs', player_id=0, season_nullable = season, context_measure_simple="FGA")
    shot_chart_data = shot_chart_data.get_data_frames()[0]
    return shot_chart_data

shot_selection_data = playoffs_shot_selection('2019-20')
shot_selection_data = shot_selection_data.append(playoffs_shot_selection('2018-19'))
shot_selection_data = shot_selection_data.append(playoffs_shot_selection('2017-18'))
shot_selection_data = shot_selection_data.append(playoffs_shot_selection('2016-17'))
shot_selection_data = shot_selection_data.append(playoffs_shot_selection('2015-16'))



def season_game_finder(season):
    season_game_data = leaguegamefinder.LeagueGameFinder(player_or_team_abbreviation = 'T', season_nullable = season)
    season_game_data = season_game_data.get_data_frames()[0]
    return season_game_data

season_game_data = season_game_finder('2019-20')
season_game_data = season_game_data.append(season_game_finder('2018-19'))
season_game_data = season_game_data.append(season_game_finder('2017-18'))
season_game_data = season_game_data.append(season_game_finder('2016-17'))
season_game_data = season_game_data.append(season_game_finder('2015-16'))
season_game_data

print('Loaded Data')

Loaded Data


## Merge Data

In [5]:
merged_shot_selection_data = pd.merge(shot_selection_data, season_game_data, how = 'left', on = ['GAME_ID', 'TEAM_ID'])
merged_shot_selection_data = merged_shot_selection_data.rename(columns={"GAME_DATE_x": "GAME_DATE"})
merged_shot_selection_data['WIN'] = np.where(merged_shot_selection_data.WL=='W', 1, 0)
merged_shot_selection_data['SEASON'] = merged_shot_selection_data.GAME_DATE.astype('str').str[:4]
merged_shot_selection_data['BUBBLE'] = np.where(merged_shot_selection_data.GAME_DATE.astype('str').str[:4] == '2020', 1, 0)
merged_shot_selection_data = merged_shot_selection_data[['ACTION_TYPE',
                                                         'SHOT_TYPE',
                                                         'SHOT_ZONE_AREA',
                                                         'SHOT_ZONE_BASIC', 
                                                         'SHOT_ZONE_RANGE', 
                                                         'SHOT_DISTANCE', 
                                                         'LOC_X', 
                                                         'LOC_Y', 
                                                         'SHOT_MADE_FLAG', 
                                                         'SEASON', 
                                                         'WIN', 
                                                         'BUBBLE']]

merged_shot_selection_data.head()

,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_AREA,SHOT_ZONE_BASIC,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_MADE_FLAG,SEASON,WIN,BUBBLE
0,Driving Floating Jump Shot,2PT Field Goal,Center(C),Mid-Range,8-16 ft.,15,-14,157,1,2020,0,1
1,Jump Shot,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,25,-107,231,0,2020,1,1
2,Jump Shot,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,26,8,262,1,2020,0,1
3,Driving Reverse Layup Shot,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,0,3,-8,1,2020,1,1
4,Jump Shot,3PT Field Goal,Right Side Center(RC),Above the Break 3,24+ ft.,27,137,236,1,2020,0,1


## Save Data to CSV

In [7]:
merged_shot_selection_data.to_csv('data/full_shot_selection_data_of_playoffs_for_past_5_years.csv', index=False)
print("Saved Data to CSV")

Saved Data to CSV
